In [1]:
import nltk 
import os 
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math 

In [2]:
pip install num2words

Note: you may need to restart the kernel to use updated packages.


In [3]:
from num2words import num2words

In [4]:
from collections import Counter

In [5]:
from nltk.corpus import stopwords

In [6]:
from nltk.tokenize import word_tokenize

In [7]:
from nltk.stem import PorterStemmer

In [32]:
title = "comp.graphics"
os.chdir("/Users/zeez/Desktop/mini_newsgroups")

In [33]:
paths = []
for (dirpath, dirnames, filenames) in os.walk(str(os.getcwd())+'/'+title+'/'):
    for i in filenames:
        paths.append(str(dirpath)+str("//")+i)

In [34]:
print(dirpath)

/Users/zeez/Desktop/mini_newsgroups/comp.graphics/


In [35]:
paths[0]

'/Users/zeez/Desktop/mini_newsgroups/comp.graphics///38402'

In [36]:
myfile = open(paths[1])
txt = myfile.read()
print(txt)
myfile.close()

Newsgroups: comp.graphics
Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!howland.reston.ans.net!ux1.cso.uiuc.edu!news.cso.uiuc.edu!ux4.cso.uiuc.edu!jroberts
From: jroberts@ux4.cso.uiuc.edu (Robertson)
Subject: ATI GUP and Graphics Wkshop/Win
Date: Wed, 21 Apr 1993 20:38:54 GMT
Message-ID: <C5upCu.8M3@news.cso.uiuc.edu>
Sender: usenet@news.cso.uiuc.edu (Net Noise owner)
Organization: University of Illinois at Urbana
Lines: 11

I have an ATI Graph. Ultra Pro VLB w/2 megs, and have a small question
about Graphics Workshop for Windows.  When I exit from it it says my
current driver can handle on 32768 colors when I am actually in 
1024x768x65000 color mode.  Is this a driver problem, a GWS error, or
what?  I am using the 1.5(59) driver under Win 3.1.  It correctly
states that I can display 16M colors when I switch to 800x600x24bit,
though.
Another question- Anybody know of any Viewers that support this card
other than Windows viewers?
Any hel

In [13]:
#Removing stop words 
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words:
            new_text = new_text + " " + w
    return np.char.strip(new_text) 

#Removing punctuation
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, " ", " ")
    data = np.char.replace(data, ',', '')
    return data 

#Convert to lowercase
def convert_lower_case(data):
    return np.char.lower(data)

#Stemming
def stemming(data):
    stemmer= PorterStemmer()

    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return np.char.strip(new_text) 

#Converting numbers to its equivalent words 
def convert_numbers(data):
    data = np.char.replace(data, "0", " zero ")
    data = np.char.replace(data, "1", " one ")
    data = np.char.replace(data, "2", " two ")
    data = np.char.replace(data, "3", " three ")
    data = np.char.replace(data, "4", " four ")
    data = np.char.replace(data, "5", " five ")
    data = np.char.replace(data, "6", " six ")
    data = np.char.replace(data, "7", " seven ")
    data = np.char.replace(data, "8", " eight ")
    data = np.char.replace(data, "9", " nine ")
    return data 

#Removing header 
def remove_header(data):
    try:
        ind = data.index('\n\n')
        data = data[ind:]
    except:
        print("No Header")
    return data 

#Removing apostrophe 
def remove_apostrophe(data):
    return np.char.replace(data, "'", "") 

#Removing single characters 
def remove_single_characters(data):
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if len(w) > 1:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

In [14]:
def preprocess(data):
    data = remove_header(data)
    data = convert_lower_case(data)
    data = convert_numbers(data)
    data = remove_punctuation(data)
    data = remove_stop_words(data)
    data = remove_apostrophe(data)
    data = remove_single_characters(data)
    data = stemming(data)
    return data

In [37]:
processed_text = []

for i in range(len(filenames)):
    file = open(dirpath+'/'+ filenames[i], 'r',
                encoding='cp1250',
                errors='ignore') 
    text = file.read().strip()
    file.close()
    
    processed_text.append(word_tokenize
                          (str(preprocess(text))))
    
    print(processed_text)

[['jgreen', 'amber', 'joe', 'green', 'write', 'anyon', 'know', 'weitek', 'nine', 'zero', 'zero', 'zero', 'graphic', 'chip', 'weitek', 'address', 'phone', 'number', 'like', 'get', 'inform', 'chip', 'ye', 'interest', 'chip', 'pleas', 'follow', 'email', 'bill', 'egan', 'cavali', 'graphic', 'houston', 'texa', 'email', 'cavali', 'blkbox', 'com']]
[['jgreen', 'amber', 'joe', 'green', 'write', 'anyon', 'know', 'weitek', 'nine', 'zero', 'zero', 'zero', 'graphic', 'chip', 'weitek', 'address', 'phone', 'number', 'like', 'get', 'inform', 'chip', 'ye', 'interest', 'chip', 'pleas', 'follow', 'email', 'bill', 'egan', 'cavali', 'graphic', 'houston', 'texa', 'email', 'cavali', 'blkbox', 'com'], ['ati', 'graph', 'ultra', 'pro', 'vlb', 'two', 'meg', 'small', 'question', 'graphic', 'workshop', 'window', 'exit', 'say', 'current', 'driver', 'handl', 'three', 'two', 'seven', 'six', 'eight', 'color', 'actual', 'one', 'zero', 'two', 'four', 'seven', 'six', 'eight', 'six', 'five', 'zero', 'zero', 'zero', 'colo

[['jgreen', 'amber', 'joe', 'green', 'write', 'anyon', 'know', 'weitek', 'nine', 'zero', 'zero', 'zero', 'graphic', 'chip', 'weitek', 'address', 'phone', 'number', 'like', 'get', 'inform', 'chip', 'ye', 'interest', 'chip', 'pleas', 'follow', 'email', 'bill', 'egan', 'cavali', 'graphic', 'houston', 'texa', 'email', 'cavali', 'blkbox', 'com'], ['ati', 'graph', 'ultra', 'pro', 'vlb', 'two', 'meg', 'small', 'question', 'graphic', 'workshop', 'window', 'exit', 'say', 'current', 'driver', 'handl', 'three', 'two', 'seven', 'six', 'eight', 'color', 'actual', 'one', 'zero', 'two', 'four', 'seven', 'six', 'eight', 'six', 'five', 'zero', 'zero', 'zero', 'color', 'mode', 'driver', 'problem', 'gw', 'error', 'use', 'one', 'five', 'five', 'nine', 'driver', 'win', 'three', 'one', 'correctli', 'state', 'display', 'one', 'six', 'color', 'switch', 'eight', 'zero', 'zero', 'six', 'zero', 'zero', 'two', 'four', 'bit', 'though', 'anoth', 'question', 'anybodi', 'know', 'viewer', 'support', 'card', 'window', 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [38]:
DF = {}
N = len(processed_text)

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
            
for i in DF:
    DF[i] = len(DF[i])
    
print(DF)

{'jgreen': 1, 'amber': 2, 'joe': 4, 'green': 2, 'write': 34, 'anyon': 30, 'know': 38, 'weitek': 1, 'nine': 43, 'zero': 60, 'graphic': 36, 'chip': 4, 'address': 12, 'phone': 6, 'number': 14, 'like': 27, 'get': 28, 'inform': 17, 'ye': 5, 'interest': 14, 'pleas': 24, 'follow': 11, 'email': 16, 'bill': 3, 'egan': 1, 'cavali': 1, 'houston': 1, 'texa': 1, 'blkbox': 1, 'com': 20, 'ati': 2, 'graph': 6, 'ultra': 2, 'pro': 2, 'vlb': 1, 'two': 66, 'meg': 1, 'small': 8, 'question': 13, 'workshop': 3, 'window': 17, 'exit': 1, 'say': 12, 'current': 11, 'driver': 8, 'handl': 10, 'three': 62, 'seven': 40, 'six': 54, 'eight': 47, 'color': 17, 'actual': 5, 'one': 76, 'four': 58, 'five': 50, 'mode': 11, 'problem': 16, 'gw': 1, 'error': 9, 'use': 42, 'win': 2, 'correctli': 2, 'state': 10, 'display': 16, 'switch': 4, 'bit': 18, 'though': 8, 'anoth': 11, 'anybodi': 14, 'viewer': 7, 'support': 13, 'card': 15, 'help': 30, 'would': 39, 'appreci': 16, 'archiv': 4, 'name': 10, 'resourc': 7, 'list': 8, 'part': 7,

In [39]:
total_vocab_size = len(DF)
print(total_vocab_size)

4865


In [40]:
total_vocab=[]
for x in DF:
    total_vocab.append(x)
    
total_vocab = [x for x in DF]
print(total_vocab[:5])

['jgreen', 'amber', 'joe', 'green', 'write']


In [41]:
def doc_freq(word):
    c = 0 
    try:
        c = DF[word]
    except:
        pass
    return c

word = "path"

print("word:", word, "-->frequency", doc_freq(word))

word: path -->frequency 2


In [42]:
doc = 0

tf_idf = {}

for i in range(N):
    
    tokens = processed_text[i]
    
    counter = Counter(tokens + processed_text[i])
    words_count = len(tokens + processed_text[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log(N+1)/(df+1)
        
        tf_idf[doc, token] = tf*idf
        
    doc+=1

In [43]:
tf_idf[(0, 'recent')]

KeyError: (0, 'recent')

In [44]:
def matching_score(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)

    query_weights = {} 
    
    for key in tf_idf:

        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
            

    query_weights = sorted(query_weights.items(), key=lambda x: x[1],reverse=True)
    
    print("")

    l = []

    for i in query_weights[:k]:
        l.append(i[0])

    print(l)

In [45]:
matching_score(1, "I recently got a file describing a library")


No Header
Matching Score

Query: I recently got a file describing a library

['recent', 'got', 'file', 'describ', 'librari']

[29]
